In [1]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [2]:
%run -i ../../core.py

In [3]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
comb_data_path = os.path.join('prover/comb_data.json')

=======================  ZK-STATS FLOW =======================

In [4]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

f_raw_input = open(data_path, "r")
data = json.loads(f_raw_input.read())["input_data"][0]
data_tensor = torch.reshape(torch.tensor(data),(1, len(data), 1))

#  dummy data for data consumer: make the bound approx same as real data
dummy_data = np.random.uniform(min(data), max(data), len(data))
json.dump({"input_data":[dummy_data.tolist()]}, open(dummy_data_path, 'w'))

dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1, len(dummy_data),1 ))
gt30_dummy_data_tensor = dummy_data_tensor[dummy_data_tensor > 30].reshape(1,-1,1)
new_dummy_size = torch.tensor(gt30_dummy_data_tensor.size()[1] )
dummy_theory_output = torch.mean(gt30_dummy_data_tensor)



In [5]:
# Verifier/ data consumer side: send desired calculation
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)
        self.new_size = nn.Parameter(data = new_dummy_size,requires_grad = False )
        self.new_X = nn.Parameter(data = gt30_dummy_data_tensor,requires_grad = False )

    def forward(self,X):
        # where part
        len_ratio =  self.new_size/X.size()[1]
        result = torch.zeros(len_ratio*X.size()[1]).reshape(1,-1,1)
        index_array = torch.tensor(range(int(len_ratio*X.size()[1])))
        for index in index_array:
            result[0][index][0] = self.new_X[0][index][0]
        check_bool = torch.sum((torch.abs(X[X>30].reshape(1,-1,1)-self.new_X)<0.01*self.new_X).double())
        print("check_bool: ", check_bool)
        if check_bool==self.new_size:
            value = torch.abs(torch.sum(result)-result.size()[1]*(self.w))<0.01*result.size()[1]*(self.w)
        else:
            value = torch.tensor(0)

        return (value, self.w)
    
verifier_define_calculation(verifier_model, verifier_model_path, [dummy_data_path])

/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_42565/3656555878.py:14: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  index_array = torch.tensor(range(int(len_ratio*X.size()[1])))
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_42565/3656555878.py:14: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  index_array = torch.tensor(range(int(len_ratio*X.size()[1])))
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_42565/3656555878.py:15: TracerWarning: Iterating over a tensor might cause the 

check_bool:  tensor(239., dtype=torch.float64)


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_42565/3656555878.py:19: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if check_bool==self.new_size:
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/onnx/utils.py:1686: UserWarning: The exported ONNX model failed ONNX shape inference. The model will not be executable by the ONNX Runtime. If this is unintended and you believe there is a bug, please report an issue at https://github.com/pytorch/pytorch/issues. Error reported by strict ONNX shape inference: [ShapeInferenceError] (op_type:ConstantOfShape, node name: /ConstantOfShape): input typestr: T1, has unsupported type: tensor(float) (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/jit/serializ

In [6]:
# prover calculates settings, send to verifier
gt30_data_tensor = data_tensor[data_tensor > 30].reshape(1,-1,1)
new_size = torch.tensor(gt30_data_tensor.size()[1] )
print("new tensor: ", gt30_data_tensor.size())
# gt30_data_tensor = torch.reshape(gt30_data_tensor_raw, (1, int(gt30_data_tensor_raw.size()[0]), 1))
theory_output = torch.mean(gt30_data_tensor)
print("Theory_output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = theory_output, requires_grad = False)
        self.new_size = nn.Parameter(data = new_size,requires_grad = False )
        self.new_X = nn.Parameter(data = gt30_data_tensor,requires_grad = False )
    def forward(self,X):
        # where part
        len_ratio =  self.new_size/X.size()[1]
        result = torch.zeros(len_ratio*X.size()[1]).reshape(1,-1,1)
        index_array = torch.tensor(range(int(len_ratio*X.size()[1])))
        for index in index_array:
            result[0][index][0] = self.new_X[0][index][0]
        check_bool = torch.sum((torch.abs(X[X>30].reshape(1,-1,1)-self.new_X)<0.01*self.new_X).double())
        print("check_bool: ", check_bool)
        if check_bool==self.new_size:
            value = torch.abs(torch.sum(result)-result.size()[1]*(self.w))<0.01*result.size()[1]*(self.w)
        else:
            value = torch.tensor(0)

        return (value, self.w)

prover_gen_settings([data_path], comb_data_path, prover_model,prover_model_path, [0], "resources", settings_path)

/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_42565/2567632164.py:19: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  index_array = torch.tensor(range(int(len_ratio*X.size()[1])))
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_42565/2567632164.py:19: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  index_array = torch.tensor(range(int(len_ratio*X.size()[1])))
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_42565/2567632164.py:20: TracerWarning: Iterating over a tensor might cause the 

new tensor:  torch.Size([1, 272, 1])
Theory_output:  tensor(52.3676)
check_bool:  tensor(272., dtype=torch.float64)


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_42565/2567632164.py:24: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if check_bool==self.new_size:


==== Generate & Calibrate Setting ====
scale:  [0]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":0,"param_scale":0,"scale_rebase_multiplier":10,"lookup_range":[0,0],"logrows":14,"num_inner_cols":1,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":300,"total_const_size":0,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,0],"model_input_scales":[0],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":[],"check_mode":"UNSAFE","version":"5.0.8","num_blinding_factors":null}


In [7]:
# Here verifier & prover can concurrently call setup since all params are public to get pk. 
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path, srs_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, comb_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path, srs_path)

==== setting up ezkl ====


spawning module 0
spawning module 2
spawning module 0
spawning module 2


Time setup: 1.3034610748291016 seconds
Theory output:  tensor(52.3676)


spawning module 0


==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 52.0
==== Generating Proof ====


spawning module 2


proof:  {'instances': [[[17970410297053518904, 9046703063145816218, 2851759239208196922, 164045840560226117], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [1460628720732602093, 15005283833837523956, 127539993950163949, 306168462079750959]]], 'proof': '257699b87b4e6b2e891c00a5b0444981b95bd65a6bb11fe332a3b85dcc4097d419e8420992c16c396b9bd4b8699e17af40a8e73bcefc79a686cf59549d16fc9f2343e9849a335fa062888793356d4bcaba18db282efaaa2522c8635e411080110c497138b98c0bd0e001a47337359f8909a6b30210b4f681568f893971e1049b0d89ca52b8cc7168d8ce111738c595b879880c236883849bcbbcd17521f5f1a708b5b1173249d3ad74b5ff28ad11af4f2c9615a3a2fd6dad762fcf220fd58524117758ca41d6f9237fdab791f995da5ef3819529edb658f6787c341b7f50802f2e72045b924b81d26fe3e7dec21bd664786dd80153dcfdf339c85f866c29982122cb174b5e638c540bbbe8ec99147c59f666ca05ef379333cfe142832e6657b1121d690375d4c20a4e35b49a2c8a80805d6dbaaa3a5a6e47929c1adc462cd7d92ae714e37ede8c2fbc570ead127426bace460980379eebb0741c49690848d64b05

In [8]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path, srs_path)

num_inputs:  1
prf instances:  [[[17970410297053518904, 9046703063145816218, 2851759239208196922, 164045840560226117], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [1460628720732602093, 15005283833837523956, 127539993950163949, 306168462079750959]]]
proof boolean:  1.0
proof result 1 : 52.0
verified
